In [1]:
pip install mss opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sounddevice soundfile moviepy


   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/31.2 MB ? eta -:--:--
    --------------------------------------- 0.5/31.2 MB 725.3 kB/s eta 0:00:43
    --------------------------------------- 0.5/31.2 MB 725.3 kB/s eta 0:00:43
   - -------------------------------------- 0.8/31.2 MB 604.4 kB/s eta 0:00:51
   - -------------------------------------- 1.0/31.2 MB 611.4 kB/s eta 0:00:50
   - -------------------------------------- 1.0/31.2 MB 611.4 kB/s eta 0:00:50
   - -------------------------------------- 1.3/31.2 MB 660.7 kB/s eta 0:00:46
   - -------------------------------------- 1.3/31.2 MB 660.7 kB/s eta 0:00:46
   - -------------------------------------- 1.3/31.2 MB 660.7 kB/s eta 0:00:46
   - -------------------------------------- 1.3/31.2 MB 660.7 kB/s eta 0:00:46
   -- ---

In [3]:
# file: phase3_screen_recorder.py
import cv2
import numpy as np
import mss
import time
import os
from datetime import datetime

def main():
    # === Settings ===
    FPS = 20                 # 20–30 is typical
    MONITOR_INDEX = 1        # 1 = primary monitor (mss indexing)
    SHOW_PREVIEW = True      # set False if you don’t want a preview window
    FOURCC = "XVID"          # "XVID" (AVI), or "MJPG" if needed

    # Prepare capture
    sct = mss.mss()
    if MONITOR_INDEX > len(sct.monitors) - 1:
        raise ValueError(f"Monitor {MONITOR_INDEX} not found. Available: {len(sct.monitors)-1}")
    monitor = sct.monitors[MONITOR_INDEX]
    width, height = monitor["width"], monitor["height"]

    # Output file name
    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    out_name = f"screen_record_{ts}.avi"

    fourcc = cv2.VideoWriter_fourcc(*FOURCC)
    out = cv2.VideoWriter(out_name, fourcc, FPS, (width, height))
    if not out.isOpened():
        raise RuntimeError("Failed to open video writer. Try changing FOURCC to 'MJPG' and/or use .avi extension.")

    if SHOW_PREVIEW:
        cv2.namedWindow("Screen Recorder", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Screen Recorder", 960, int(960 * height / width))

    print(f"[INFO] Recording {width}x{height} at {FPS} FPS")
    print(f"[INFO] Saving to: {os.path.abspath(out_name)}")
    print("[INFO] Press 'q' in the preview window to stop.")

    frame_interval = 1.0 / FPS
    next_time = time.time()

    try:
        while True:
            # Throttle to target FPS
            now = time.time()
            if now < next_time:
                time.sleep(next_time - now)
            next_time = time.time() + frame_interval

            # Capture screen
            img = np.array(sct.grab(monitor))   # BGRA
            frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            # Optional timestamp overlay
            cv2.putText(frame, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            out.write(frame)

            if SHOW_PREVIEW:
                cv2.imshow("Screen Recorder", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    except KeyboardInterrupt:
        print("\n[INFO] Stopped via KeyboardInterrupt.")

    finally:
        out.release()
        if SHOW_PREVIEW:
            cv2.destroyAllWindows()
        sct.close()
        print("[INFO] Recording finished.")

if __name__ == "__main__":
    main()


[INFO] Recording 1920x1080 at 20 FPS
[INFO] Saving to: C:\Users\Suraj Garole\screen_record_2025-09-27_01-03-07.avi
[INFO] Press 'q' in the preview window to stop.
[INFO] Recording finished.


In [4]:
pip install sounddevice soundfile moviepy


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install moviepy


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install moviepy sounddevice soundfile


Note: you may need to restart the kernel to use updated packages.


In [9]:
import sys
print(sys.executable)


C:\Users\Suraj Garole\Conda\python.exe


In [2]:
import sys
import subprocess

# Upgrade pip, setuptools, wheel
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])

# Install required packages
subprocess.check_call([sys.executable, "-m", "pip", "install", "moviepy", "imageio-ffmpeg"])


0

In [3]:
import cv2
import numpy as np
import mss
import time
import os
from datetime import datetime
import threading
import sounddevice as sd
import soundfile as sf

# Try to import moviepy; fall back gracefully if not installed
try:
    from moviepy.editor import VideoFileClip, AudioFileClip
    MOVIEPY_AVAILABLE = True
except Exception as e:
    MOVIEPY_AVAILABLE = False
    print("[WARN] moviepy not available:", e)
    print("       Install inside your active interpreter:")
    print("       pip install moviepy imageio-ffmpeg")

def record_screen(stop_event, out_path, fps=20, monitor_index=1, fourcc="XVID", show_preview=True):
    sct = mss.mss()
    if monitor_index > len(sct.monitors) - 1:
        raise ValueError(f"Monitor {monitor_index} not found. Available: {len(sct.monitors)-1}")
    monitor = sct.monitors[monitor_index]
    width, height = monitor["width"], monitor["height"]

    writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*fourcc), fps, (width, height))
    if not writer.isOpened():
        raise RuntimeError("Failed to open video writer. Try fourcc='MJPG' and .avi output.")

    if show_preview:
        cv2.namedWindow("Screen Recorder", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Screen Recorder", 960, int(960 * height / width))

    frame_interval = 1.0 / fps
    next_time = time.time()

    try:
        while not stop_event.is_set():
            # throttle to target FPS
            now = time.time()
            if now < next_time:
                time.sleep(max(0, next_time - now))
            next_time = time.time() + frame_interval

            # capture frame
            img = np.array(sct.grab(monitor))   # BGRA
            frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            # timestamp overlay
            cv2.putText(frame, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            writer.write(frame)

            if show_preview:
                cv2.imshow("Screen Recorder", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    stop_event.set()
    finally:
        writer.release()
        if show_preview:
            cv2.destroyAllWindows()
        sct.close()

def record_mic(stop_event, wav_path, sr=16000, channels=1):
    # Records microphone audio to WAV until stop_event is set
    with sf.SoundFile(wav_path, mode='w', samplerate=sr, channels=channels) as file:
        def callback(indata, frames, time_info, status):
            if status:
                print("[Audio Warning]", status)
            file.write(indata.copy())

        with sd.InputStream(samplerate=sr, channels=channels, callback=callback):
            while not stop_event.is_set():
                time.sleep(0.05)

def mux_video_audio(video_path, audio_path, out_mp4_path):
    if not MOVIEPY_AVAILABLE:
        raise RuntimeError("moviepy not available — cannot mux automatically.")
    # Merge video and audio into MP4
    with VideoFileClip(video_path) as v, AudioFileClip(audio_path) as a:
        v = v.set_audio(a)
        # libx264 + aac are widely supported
        v.write_videofile(out_mp4_path, codec="libx264", audio_codec="aac")

def main():
    # SETTINGS
    FPS = 20
    MONITOR_INDEX = 1   # 1 = primary monitor
    FOURCC = "XVID"     # or "MJPG"
    SHOW_PREVIEW = True
    SAMPLERATE = 16000  # mic sample rate

    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    raw_video = f"screen_raw_{ts}.avi"
    raw_audio = f"screen_audio_{ts}.wav"
    final_mp4 = f"screen_record_{ts}.mp4"

    print("[INFO] Starting screen + mic recording.")
    print("[INFO] Stop by pressing 'q' in the preview window (or Ctrl+C in terminal).")

    stop_event = threading.Event()

    t_video = threading.Thread(
        target=record_screen,
        args=(stop_event, raw_video),
        kwargs=dict(fps=FPS, monitor_index=MONITOR_INDEX, fourcc=FOURCC, show_preview=SHOW_PREVIEW),
        daemon=True
    )
    t_audio = threading.Thread(
        target=record_mic,
        args=(stop_event, raw_audio),
        kwargs=dict(sr=SAMPLERATE, channels=1),
        daemon=True
    )

    t_video.start()
    t_audio.start()

    try:
        while not stop_event.is_set():
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\n[INFO] Stopping via KeyboardInterrupt.")
        stop_event.set()

    t_video.join()
    t_audio.join()

    if MOVIEPY_AVAILABLE:
        print("[INFO] Muxing video + audio into MP4…")
        mux_video_audio(raw_video, raw_audio, final_mp4)
        print(f"[DONE] Saved: {os.path.abspath(final_mp4)}")
    else:
        print("[INFO] MoviePy unavailable. Your raw files are ready:")
        print(f"       Video: {os.path.abspath(raw_video)}")
        print(f"       Audio: {os.path.abspath(raw_audio)}")
        print("       To enable automatic MP4 export, install:")
        print("       pip install moviepy imageio-ffmpeg")
        print("       Then run again.")

if __name__ == "__main__":
    main()







import cv2
import numpy as np
import mss
import time
import os
from datetime import datetime
import threading
import sounddevice as sd
import soundfile as sf

try:
    from moviepy.editor import VideoFileClip, AudioFileClip
    MOVIEPY_AVAILABLE = True
except Exception as e:
    MOVIEPY_AVAILABLE = False
    print("[WARN] moviepy not available:", e)
    print("       pip install moviepy imageio-ffmpeg")

def record_screen(stop_event, out_path, fps=20, capture_region=None, fourcc="XVID", show_preview=True):
    """
    capture_region: tuple (left, top, width, height) for single window,
                    or None to capture full primary monitor.
    """
    sct = mss.mss()
    if capture_region is None:
        monitor = sct.monitors[1]  # primary monitor
    else:
        left, top, width, height = capture_region
        monitor = {"left": left, "top": top, "width": width, "height": height}

    width, height = monitor["width"], monitor["height"]
    writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*fourcc), fps, (width, height))
    if not writer.isOpened():
        raise RuntimeError("Failed to open video writer. Try fourcc='MJPG' and .avi output.")

    if show_preview:
        cv2.namedWindow("Screen Recorder", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Screen Recorder", 960, int(960 * height / width))

    frame_interval = 1.0 / fps
    next_time = time.time()

    try:
        while not stop_event.is_set():
            now = time.time()
            if now < next_time:
                time.sleep(max(0, next_time - now))
            next_time = time.time() + frame_interval

            img = np.array(sct.grab(monitor))  # BGRA
            frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            # timestamp overlay
            cv2.putText(frame, datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            writer.write(frame)

            if show_preview:
                # Shrink preview so it doesn't overlap recorded area
                preview = cv2.resize(frame, (640, int(640 * height / width)))
                cv2.imshow("Screen Recorder", preview)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    stop_event.set()
    finally:
        writer.release()
        if show_preview:
            cv2.destroyAllWindows()
        sct.close()

def record_mic(stop_event, wav_path, sr=16000, channels=1):
    with sf.SoundFile(wav_path, mode='w', samplerate=sr, channels=channels) as file:
        def callback(indata, frames, time_info, status):
            if status:
                print("[Audio Warning]", status)
            file.write(indata.copy())

        with sd.InputStream(samplerate=sr, channels=channels, callback=callback):
            while not stop_event.is_set():
                time.sleep(0.05)

def mux_video_audio(video_path, audio_path, out_mp4_path):
    if not MOVIEPY_AVAILABLE:
        raise RuntimeError("moviepy not available — cannot mux automatically.")
    with VideoFileClip(video_path) as v, AudioFileClip(audio_path) as a:
        v = v.set_audio(a)
        v.write_videofile(out_mp4_path, codec="libx264", audio_codec="aac")

def main():
    FPS = 20
    FOURCC = "XVID"
    SHOW_PREVIEW = True
    SAMPLERATE = 16000

    # Change this to your window's coordinates
    CAPTURE_REGION = (200, 100, 1280, 720)  # left, top, width, height

    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    raw_video = f"screen_raw_{ts}.avi"
    raw_audio = f"screen_audio_{ts}.wav"
    final_mp4 = f"screen_record_{ts}.mp4"

    print("[INFO] Starting single-window recording.")
    print("[INFO] Stop by pressing 'q' in the preview window or Ctrl+C in terminal.")

    stop_event = threading.Event()

    t_video = threading.Thread(
        target=record_screen,
        args=(stop_event, raw_video),
        kwargs=dict(fps=FPS, capture_region=CAPTURE_REGION, fourcc=FOURCC, show_preview=SHOW_PREVIEW),
        daemon=True
    )
    t_audio = threading.Thread(
        target=record_mic,
        args=(stop_event, raw_audio),
        kwargs=dict(sr=SAMPLERATE, channels=1),
        daemon=True
    )

    t_video.start()
    t_audio.start()

    try:
        while not stop_event.is_set():
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\n[INFO] Stopping via KeyboardInterrupt.")
        stop_event.set()

    t_video.join()
    t_audio.join()

    if MOVIEPY_AVAILABLE:
        print("[INFO] Muxing video + audio into MP4…")
        mux_video_audio(raw_video, raw_audio, final_mp4)
        print(f"[DONE] Saved: {os.path.abspath(final_mp4)}")
    else:
        print("[INFO] MoviePy unavailable. Your raw files are ready:")
        print(f"       Video: {os.path.abspath(raw_video)}")
        print(f"       Audio: {os.path.abspath(raw_audio)}")

if __name__ == "__main__":
    main()


[WARN] moviepy not available: No module named 'moviepy.editor'
       Install inside your active interpreter:
       pip install moviepy imageio-ffmpeg
[INFO] Starting screen + mic recording.
[INFO] Stop by pressing 'q' in the preview window (or Ctrl+C in terminal).
[INFO] MoviePy unavailable. Your raw files are ready:
       Video: C:\Users\Suraj Garole\screen_raw_2025-09-27_01-19-16.avi
       Audio: C:\Users\Suraj Garole\screen_audio_2025-09-27_01-19-16.wav
       To enable automatic MP4 export, install:
       pip install moviepy imageio-ffmpeg
       Then run again.
[WARN] moviepy not available: No module named 'moviepy.editor'
       pip install moviepy imageio-ffmpeg
[INFO] Starting single-window recording.
[INFO] Stop by pressing 'q' in the preview window or Ctrl+C in terminal.
[INFO] MoviePy unavailable. Your raw files are ready:
       Video: C:\Users\Suraj Garole\screen_raw_2025-09-27_01-19-30.avi
       Audio: C:\Users\Suraj Garole\screen_audio_2025-09-27_01-19-30.wav


In [4]:
pip install moviepy imageio-ffmpeg


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install moviepy imageio-ffmpeg


Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
!"{sys.executable}" -m pip install pyautogui


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): s

  DEPRECATION: Building 'pygetwindow' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pygetwindow'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'pytweening' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pytweening'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'mouseinfo' using the legacy setup.py bdist_wh

In [7]:
import pyautogui
import mouseinfo
import pygetwindow

print("PyAutoGUI and dependencies are working!")


PyAutoGUI and dependencies are working!


In [3]:
import cv2
import numpy as np
import pyautogui
import sounddevice as sd
import wave
import threading
import subprocess
import os
import time
from datetime import datetime

# ===== SETTINGS =====
FRAME_RATE = 20.0
VIDEO_FILENAME = f"screen_raw_{datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
AUDIO_FILENAME = f"screen_audio_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
FINAL_FILENAME = f"screen_record_{datetime.now().strftime('%Y%m%d_%H%M%S')}_final.mp4"

# Adjust this region to your desired window or full screen
monitor = {"top": 100, "left": 100, "width": 1280, "height": 720}

# ===== AUDIO RECORDING FUNCTION =====
def record_audio(filename, duration, samplerate=44100, channels=2):
    print("[INFO] Recording audio...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=channels, dtype='int16')
    sd.wait()
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(2)
        wf.setframerate(samplerate)
        wf.writeframes(audio.tobytes())
    print("[INFO] Audio saved:", filename)

# ===== SCREEN RECORDING FUNCTION =====
def record_screen(filename, duration):
    print("[INFO] Recording screen...")
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(filename, fourcc, FRAME_RATE, (monitor["width"], monitor["height"]))

    cv2.namedWindow("Recording Preview (Press Q to stop)", cv2.WINDOW_NORMAL)

    start_time = time.time()
    while time.time() - start_time < duration:
        img = pyautogui.screenshot(region=(monitor["left"], monitor["top"], monitor["width"], monitor["height"]))
        frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        out.write(frame)

        preview = cv2.resize(frame, (640, 360))
        cv2.imshow("Recording Preview (Press Q to stop)", preview)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    out.release()
    cv2.destroyAllWindows()
    print("[INFO] Video saved:", filename)

# ===== MERGE VIDEO + AUDIO =====
def merge_audio_video(video_file, audio_file, final_file):
    print("[INFO] Merging audio and video using FFmpeg...")
    if os.system("ffmpeg -version") != 0:
        print("[ERROR] FFmpeg not found! Install FFmpeg and make sure it's in PATH.")
        print(f"Manual merge command:\nffmpeg -i \"{video_file}\" -i \"{audio_file}\" -c:v libx264 -preset veryfast -crf 23 -c:a aac -b:a 160k \"{final_file}\"")
        return
    subprocess.run([
        "ffmpeg", "-y",
        "-i", video_file,
        "-i", audio_file,
        "-c:v", "libx264",
        "-preset", "veryfast",
        "-crf", "23",
        "-c:a", "aac",
        "-b:a", "160k",
        final_file
    ])
    print("[INFO] Final video saved as:", final_file)

# ===== MAIN =====
if __name__ == "__main__":
    duration = 20  # seconds, adjust as needed

    audio_thread = threading.Thread(target=record_audio, args=(AUDIO_FILENAME, duration))
    video_thread = threading.Thread(target=record_screen, args=(VIDEO_FILENAME, duration))

    audio_thread.start()
    video_thread.start()

    audio_thread.join()
    video_thread.join()

    merge_audio_video(VIDEO_FILENAME, AUDIO_FILENAME, FINAL_FILENAME)


[INFO] Recording audio...
[INFO] Recording screen...
[INFO] Video saved: screen_raw_20250927_155320.avi
[INFO] Audio saved: screen_audio_20250927_155320.wav
[INFO] Merging audio and video using FFmpeg...
[ERROR] FFmpeg not found! Install FFmpeg and make sure it's in PATH.
Manual merge command:
ffmpeg -i "screen_raw_20250927_155320.avi" -i "screen_audio_20250927_155320.wav" -c:v libx264 -preset veryfast -crf 23 -c:a aac -b:a 160k "screen_record_20250927_155320_final.mp4"
